In [10]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [11]:
from ipygallery import Widget
Widget()

Widget()

In [21]:
from typing import List

# you can write to stdout for debugging purposes, e.g.
# print("this is a debug message")


# Return your counterexample here.
def generate_counterexample() -> List[int]:
    prices = [5, 1, 3, 2, 4]
    return prices


# Fix the function below, so it does not contain a bug.
def solution(prices: List[int]) -> int:
    N = len(prices)
    holding = True  # start holding one asset
    total_profit = 0

    for i in range(1, N):
        # Detect local peak: price[i] > price[i-1] and (i == N-1 or price[i] >= price[i+1])
        if holding and prices[i] > prices[i-1] and (i == N - 1 or prices[i] >= prices[i + 1]):
            total_profit += prices[i]
            holding = False
        # Detect local valley: price[i] < price[i-1] and (i == N-1 or price[i] <= prices[i+1])
        elif not holding and prices[i] < prices[i-1] and (i == N - 1 or prices[i] <= prices[i + 1]):
            total_profit -= prices[i]
            holding = True

    # If still holding asset at end, sell at last price
    if holding:
        total_profit += prices[-1]

    return total_profit

print("Counterexample:", generate_counterexample())
print("Solution:", solution(generate_counterexample()))


Counterexample: [5, 1, 3, 2, 4]
Solution: 5


In [ ]:
import json

from django.db import models
from django.http import HttpRequest
from django.http import HttpResponse
from django.http import JsonResponse


class Product(models.Model):
    name = models.CharField(max_length=15, primary_key=True)
    price = models.IntegerField()
    quantity = models.IntegerField()


def shopping(request: HttpRequest) -> HttpResponse:
    # Only allow GET
    if request.method != "GET":
        return JsonResponse({"error": "Method not allowed"}, status=405)

    # Validate budget parameter
    try:
        budget = int(request.GET.get("budget", ""))
        if budget < 0:
            raise ValueError
    except ValueError:
        return JsonResponse({"error": "Invalid or missing budget"}, status=400)

    # Fetch all products sorted by price ascending (greedy approach)
    products = Product.objects.all().order_by("price")

    remaining = budget
    result = []

    for product in products:
        max_affordable_qty = remaining // product.price
        qty_to_buy = min(product.quantity, max_affordable_qty)
        if qty_to_buy > 0:
            result.append({
                "name": product.name,
                "quantity": qty_to_buy
            })
            remaining -= qty_to_buy * product.price
        if remaining <= 0:
            break

    return JsonResponse(result)
